In [ ]:
import torch
import numpy as np
import pickle
from music21 import instrument, note, stream, chord
from train import MusicLSTM

# Load mappings
with open("data/mappings.pkl", "rb") as f:
    note_to_int, pitchnames = pickle.load(f)

int_to_note = {num: note for note, num in note_to_int.items()}

# Prepare model
model = MusicLSTM(1, 256, len(pitchnames))
model.load_state_dict(torch.load("model.pth"))
model.eval()

# Generate sequence
sequence_length = 100
start = np.random.randint(0, len(pitchnames)-sequence_length)
pattern = [note_to_int[note] for note in pitchnames[start:start+sequence_length]]

prediction_output = []

for _ in range(200):  # generate 200 notes
    input_seq = torch.tensor(np.reshape(pattern, (1, sequence_length, 1)) / float(len(pitchnames)), dtype=torch.float32)
    prediction = model(input_seq).detach().numpy()
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:]

# Convert to MIDI
offset = 0
output_notes = []

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = [note.Note(int(n)) for n in notes_in_chord]
        for n in notes:
            n.storedInstrument = instrument.Piano()
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='generated_music.mid')
print("Music generated: generated_music.mid")
